In [1]:
import pandas as pd
file=pd.read_csv('alltweetscopy.csv')

In [2]:
sample=file

# Decoding

In [3]:
import unidecode,codecs
def decoding(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    #decoded=html.unescape(text)
    return decoded

In [4]:
import warnings
warnings.filterwarnings("ignore")
sample['decoded']=sample['text'].apply(lambda x: decoding(x))

# Lower Case

In [5]:
sample['lowerCase']=sample['decoded'].apply(lambda x: x.lower())

# Aphostrophe Handling

In [6]:

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 


In [7]:
import re
def apostrophe_handling(text):
    apostrophe_handled = re.sub("’", "'", text)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    return expanded

In [8]:
sample['apos_handling']=sample['lowerCase'].apply(lambda x: apostrophe_handling(x))

# Remove @mention

In [9]:
sample['at_mention']=sample['apos_handling'].apply(lambda x: " ".join(x for x in x.split() if not (x.startswith('@') 
                                                                     or x.startswith('&'))))

# Remove all Other

In [10]:
import re,string
def cleaning(text):
    #space cleaning
    p1 = re.compile('(%s)' % ('|'.join([c for c in string.whitespace if not c == ' '])))
    p2 = re.compile(' +')
    cleaned_space=p2.sub(' ', p1.sub('', text))
    #links
    cleaned_links = re.sub(r'http\S+', '', cleaned_space)
    #punctuation
    cleaned_punc=''
    for i in cleaned_links:
        if i in string.punctuation:
            pass
        else:
            cleaned_punc+=i
    #numbers
    result = " ".join(i for i in cleaned_punc.split() if not i.isdigit())
    return result

In [11]:
sample['all_remove']=sample['at_mention'].apply(lambda x: cleaning(x))

# Stop Words

In [12]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
sample['stopwords']=sample['all_remove'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Lemmatize

In [13]:
from textblob import Word
sample['lemmatize'] = sample['stopwords'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# Sentiment Aanalyses using VADER

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [15]:
def sentiment(text):
    analyzer=SentimentIntensityAnalyzer()
    new_dict={}
    score=analyzer.polarity_scores(text)
    for key in score:
        if key!='compound':
            new_dict[key]=score[key]
    return new_dict

In [16]:
sample['sentiment_lemma']=sample['lemmatize'].apply(lambda x: sentiment(x))

In [17]:
sample['sentiment_text']=sample['text'].apply(lambda x: sentiment(x))

In [18]:
sample

,text,decoded,lowerCase,apos_handling,at_mention,all_remove,stopwords,lemmatize,sentiment_lemma,sentiment_text
0,I have bad ducking anxiety I need to blaze lol...,I have bad ducking anxiety I need to blaze lol...,i have bad ducking anxiety i need to blaze lol...,i have bad ducking anxiety i need to blaze lol...,i have bad ducking anxiety i need to blaze lol...,i have bad ducking anxiety i need to blaze lol...,bad ducking anxiety need blaze lol stonerprobs,bad ducking anxiety need blaze lol stonerprobs,"{'neg': 0.433, 'neu': 0.333, 'pos': 0.233}","{'neg': 0.371, 'neu': 0.429, 'pos': 0.2}"
1,now you\u2019ve swum in lakes with radiation! ...,now you've swum in lakes with radiation! chara...,now you've swum in lakes with radiation! chara...,now you have swum in lakes with radiation! cha...,now you have swum in lakes with radiation! cha...,now you have swum in lakes with radiation char...,swum lakes radiation character development growth,swum lake radiation character development growth,"{'neg': 0.0, 'neu': 0.658, 'pos': 0.342}","{'neg': 0.0, 'neu': 0.758, 'pos': 0.242}"
2,@ulllom Personally,@ulllom Personally,@ulllom personally,@ulllom personally,personally,personally,personally,personally,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0}","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0}"
3,In my fatigue,In my fatigue,in my fatigue,in my fatigue,in my fatigue,in my fatigue,fatigue,fatigue,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0}","{'neg': 0.5, 'neu': 0.5, 'pos': 0.0}"
4,RT @destinydc1: ASS (and I can\u2019t stress t...,RT @destinydc1: ASS (and I can't stress this e...,rt @destinydc1: ass (and i can't stress this e...,rt @destinydc1: ass (and i cannot stress this ...,rt ass (and i cannot stress this enough) whoop...,rt ass and i cannot stress this enough whoopen,rt ass cannot stress enough whoopen,rt as cannot stress enough whoopen,"{'neg': 0.0, 'neu': 0.682, 'pos': 0.318}","{'neg': 0.458, 'neu': 0.542, 'pos': 0.0}"
5,RT @cubewatermelon: Unhappy Refrain doodle~ Th...,RT @cubewatermelon: Unhappy Refrain doodle~ Th...,rt @cubewatermelon: unhappy refrain doodle~ th...,rt @cubewatermelon: unhappy refrain doodle~ th...,rt unhappy refrain doodle~ thank you wowaka fo...,rt unhappy refrain doodle thank you wowaka for...,rt unhappy refrain doodle thank wowaka express...,rt unhappy refrain doodle thank wowaka express...,"{'neg': 0.389, 'neu': 0.284, 'pos': 0.327}","{'neg': 0.308, 'neu': 0.489, 'pos': 0.203}"
6,I fucking (and I can\u2019t stress this enough...,I fucking (and I can't stress this enough) lov...,i fucking (and i can't stress this enough) lov...,i fucking (and i cannot stress this enough) lo...,i fucking (and i cannot stress this enough) lo...,i fucking and i cannot stress this enough love...,fucking cannot stress enough love,fucking cannot stress enough love,"{'neg': 0.375, 'neu': 0.334, 'pos': 0.291}","{'neg': 0.231, 'neu': 0.452, 'pos': 0.317}"
7,RT @Jom16943754: @RahulGandhi sir please ....,RT @Jom16943754: @RahulGandhi sir please ....,rt @jom16943754: @rahulgandhi sir please ....,rt @jom16943754: @rahulgandhi sir please ....,rt sir please ...har ek government exams ki ag...,rt sir please har ek government exams ki age l...,rt sir please har ek government exams ki age l...,rt sir please har ek government exam ki age li...,"{'neg': 0.0, 'neu': 0.892, 'pos': 0.108}","{'neg': 0.0, 'neu': 0.909, 'pos': 0.091}"
8,i stress myself out,i stress myself out,i stress myself out,i stress myself out,i stress myself out,i stress myself out,stress,stress,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0}","{'neg': 0.583, 'neu': 0.417, 'pos': 0.0}"
9,Big L &amp; Shyheim - Furious Anger https:\/\/...,Big L &amp; Shyheim - Furious Anger https:\/\/...,big l &amp; shyheim - furious anger https:\/\/...,big l &amp; shyheim - furious anger https:\/\/...,big l shyheim - furious anger https:\/\/t.co\/...,big l shyheim furious anger via,big l shyheim furious anger via,big l shyheim furious anger via,"{'neg': 0.712, 'neu': 0.288, 'pos': 0.0}","{'neg': 0.552, 'neu': 0.448, 'pos': 0.0}"


In [23]:
text = " ".join(word for word in sample.lemmatize)
print ("There are {} words in the combination of all review.".format(len(text)))

There are 637852 words in the combination of all review.


In [26]:
b'Craving some pussy or dick! \\xf0\\x9f\\x98\\x8f\\xf0\\x9f\\x98\\x89\\xf0\\x9f\\x98\\x8d\\xf0\\x9f\\x98\\xbb\\xf0\\x9f\\x91\\x85\\xf0\\x9f\\x92\\xa6\\xf0\\x9f\\x92\\xa6\\n#lonely #pussylicking #teennude #finger #cumslut #tittytuesday #masturbation\\xe2\\x80\\xa6 https://t.co/VdWOE7MXlo'.decode('UTF-8')

'Craving some pussy or dick! \\xf0\\x9f\\x98\\x8f\\xf0\\x9f\\x98\\x89\\xf0\\x9f\\x98\\x8d\\xf0\\x9f\\x98\\xbb\\xf0\\x9f\\x91\\x85\\xf0\\x9f\\x92\\xa6\\xf0\\x9f\\x92\\xa6\\n#lonely #pussylicking #teennude #finger #cumslut #tittytuesday #masturbation\\xe2\\x80\\xa6 https://t.co/VdWOE7MXlo'

In [31]:
'\uFEFF'.encode('UTF-8')

b'\xef\xbb\xbf'

In [33]:
b'\xef\xbb\xbf'.decode('UTF-8')

'\ufeff'